In [12]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from sklearn.datasets import make_spd_matrix

In [71]:
# I'm pretty sure this part is right. I blame any mistakes on Abdou.
def HFUL(y, X, Z):
    n = y.shape[0]
    k = X.shape[1]
    P = Z @ np.linalg.inv(Z.T @ Z) @ Z.T
    X_bar = np.column_stack((y, X))
    # Calculate alpha hat
    PXbarXbar = np.zeros((X_bar.shape[1], X_bar.shape[1]))
    for i in range(n):
        PXbarXbar = PXbarXbar + P[i, i] * np.outer(X_bar[i, :], X_bar[i, :])
    A = np.linalg.inv(X_bar.T @ X_bar) @ (X_bar.T @ P @ X_bar - PXbarXbar)
    w, v = np.linalg.eig(A)
    alpha_tilde = np.amin(w)
    alpha_hat = ((n+1) * alpha_tilde - 1)/(n + alpha_tilde - 1)
    # Now calculate b HFUL
    PXX = np.zeros((X.shape[1], X.shape[1]))
    PXy = np.zeros(k)
    for i in range(n):
        PXX = PXX + P[i, i] * np.outer(X[i, :], X[i, :])
    for i in range(n):
        PXy = PXy + P[i, i] * X[i, :] * y[i]
    b_hful_term1 = X.T @ P @ X - PXX - alpha_hat * X.T @ X
    b_hful_term2 = X.T @ P @ y - PXy - alpha_hat * X.T @ y
    return np.linalg.inv(b_hful_term1) @ b_hful_term2

In [97]:
""" I know this part has problems. The covariance matrix must be positive definite. I don't want to have to specify every
term of the covariance matrix, so I just let it be a random pos def matrix (using make_spd_matrix). But then I changed 
some of the terms to make sure that my z variables are uncorrelated with the error term u. After doing this, the matrix
is only sometimes positive definite. When an error pops up, I just keep retrying until I get lucky and it's still
positive definite."""
# Generate data, use random draws from multivariate normal
n = 10000
k = 3
l = 50
dim_cov_matrix = k + l + 1
cov_matrix = make_spd_matrix(dim_cov_matrix) # initialize covariance matrix as random
for i in range(k, l):
    cov_matrix[i, (k+l)] = 0 # make sure exclusion restriction holds
    cov_matrix[(k+l), i] = 0
data_dist = multivariate_normal(cov=cov_matrix)
data = data_dist.rvs(n)
X = data[:, range(0, k)]
Z = data[:, range(k, l)]
u = data[:, k+l]

In [98]:
# Create a random beta. Print it so I know the truth. Then generate my y vector. 
beta = np.random.rand(k)
print(beta)
y = X @ beta + u

[0.32970534 0.17514681 0.39123993]


In [99]:
# Estimate b_HFUL using the function above. Compare what it spits out to the true beta printed above.
b = HFUL(y, X, Z)
b

array([0.33531128, 0.18853401, 0.3969155 ])